In [ ]:
import pandas as pd
import joblib
import ipywidgets as widgets
from IPython.display import display, clear_output
from io import StringIO
import base64
import os

# Load model
model_path = "model/no_show_model.pkl"
model = joblib.load(model_path)

# Header
display(widgets.HTML("<h2 style='color: #2c3e50;'>🧠 Medical Appointment No-Show Predictor</h2>"))
display(widgets.HTML("<p>Upload a CSV file of patient appointment data and get predictions below.</p>"))

# File upload widget
uploader = widgets.FileUpload(
    accept=".csv",
    multiple=False,
    description="📤 Upload CSV"
)

output = widgets.Output()
display(uploader, output)

def on_upload_change(change):
    output.clear_output()
    if uploader.value:
        uploaded_file = list(uploader.value.values())[0]
        content = uploaded_file['content']
        try:
            df = pd.read_csv(StringIO(content.decode('utf-8')))

            with output:
                print("✅ File uploaded successfully. Previewing data:")
                display(df.head())

                # Preprocessing
                df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'], errors='coerce')
                df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'], errors='coerce')
                df['waiting_days'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days
                df['Gender'] = df['Gender'].map({'F': 0, 'M': 1})
                df['scheduled_dayofweek'] = df['ScheduledDay'].dt.dayofweek
                df['appointment_dayofweek'] = df['AppointmentDay'].dt.dayofweek
                df['same_day'] = (df['waiting_days'] == 0).astype(int)
                df['age_group'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], labels=[0, 1, 2, 3, 4])

                df.drop(columns=['PatientId', 'AppointmentID', 'ScheduledDay', 'AppointmentDay'], inplace=True)

                df = pd.get_dummies(df, columns=['Neighbourhood', 'age_group'], drop_first=True)

                expected_cols = model.get_booster().feature_names
                for col in expected_cols:
                    if col not in df.columns:
                        df[col] = 0
                df = df[expected_cols]

                df['No-Show Probability'] = model.predict_proba(df)[:, 1]
                df['Prediction'] = model.predict(df)
                df['Prediction Label'] = df['Prediction'].map({0: "✅ Will Show", 1: "❌ No-Show Likely"})

                print("\n📊 Prediction Results:")
                display(df[['No-Show Probability', 'Prediction Label']])

                # CSV download link
                csv_data = df.to_csv(index=False)
                b64 = base64.b64encode(csv_data.encode()).decode()
                href = f'<a download="no_show_predictions.csv" href="data:text/csv;base64,{b64}" target="_blank">⬇ Download Full Prediction CSV</a>'
                display(widgets.HTML(value=href))

        except Exception as e:
            with output:
                print(f"❌ Error: {e}")

uploader.observe(on_upload_change, names='value')
